In [1]:
import sys
import time

sys.path.append("..")
import uniplot as up
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import solv_fzcp as sfzcp
import uvarprob as uvpr

In [9]:
test_file = "../test_casado.csv"

In [10]:
# Maximal number of steps
max_steps = 1e6
# Tolerance
epsilon = 1e-2
# If True - global Lipschitz constant is used
global_lipschitz_interval = True
# If True - the record value is taken from the test database
known_record = False
# How many points to skip in vizualization - regular step
skip = 1000
# The size of the legend in plots
legend_size = 2

In [11]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col='name', comment='#')
    return data

In [12]:
df = read_problems(test_file)
df

,objective,a,b,min_f,min_x
name,,,,,
1,-0.5*x^2*(log(x))+5,0.2,7.0,0,0
2,-exp(-x)*sin(2*pi*x)+1,0.2,7.0,0,0
3,-(sqrt(x))*sin(x)+1,0.2,7.0,0,0
4,x*sin(x)+sin(10.0*x/3.0)+log(x)-0.84*x+1.3,0.2,7.0,0,0
5,x+sin(5.0*x),0.2,7.0,0,0
6,-x*sin(x)+5,0.2,7.0,0,0
7,sin(x)*cos(x)-1.5*(sin(x))^2.0+1.2,0.2,7.0,0,0
8,2*cos(x)+cos(2*x)+5,0.2,7.0,0,0
9,2.0*sin(x)*exp(-x),0.2,7.0,0,0


In [13]:
points_db = {}
psl_lipint_points_list = []
psl_lip_points_list = []
psqe_lipint_points_list = []
psqe_lip_points_list = []
bnb2_lipint_points_list = []
bnb2_lip_points_list = []

In [14]:
def log_point(x, points_list):
    points_list.append(x)

In [17]:
from FZCP.test import test_last

test_last(df, 1e-5, repeat=100)

test.Index,PL_N,PI_N,QL_N,QI_N,PL_R,PI_R,QL_R,QI_R
1 & 0.188 & 0.253 & 0.179 & 0.127 & 0.115 & 0.151 & 0.117 & 0.036 & 0.034 \\
2 & 0.020 & 0.243 & 0.202 & 0.179 & 0.165 & 0.104 & 0.092 & 0.100 & 0.094 \\
3 & 0.190 & 0.277 & 0.231 & 0.159 & 0.146 & 0.164 & 0.139 & 0.075 & 0.070 \\
4 & 0.456 & 0.339 & 0.263 & 0.215 & 0.182 & 0.225 & 0.153 & 0.098 & 0.078 \\
5 & 0.246 & 0.178 & 0.117 & 0.112 & 0.100 & 0.129 & 0.074 & 0.036 & 0.033 \\
6 & 0.029 & 0.056 & 0.044 & 0.031 & 0.027 & 0.030 & 0.026 & 0.022 & 0.020 \\
7 & 0.359 & 0.220 & 0.189 & 0.182 & 0.170 & 0.162 & 0.137 & 0.068 & 0.064 \\
8 & 0.017 & 0.041 & 0.032 & 0.027 & 0.024 & 0.023 & 0.020 & 0.026 & 0.024 \\
9 & 0.181 & 1.498 & 1.245 & 0.171 & 0.157 & 0.694 & 0.589 & 0.095 & 0.087 \\
10 & 0.357 & 1.454 & 1.197 & 0.321 & 0.292 & 0.641 & 0.534 & 0.156 & 0.143 \\
11 & 1.822 & 318.990 & 59.998 & 4.652 & 1.334 & 183.905 & 29.252 & 2.501 & 0.645 \\
12 & 0.020 & 0.322 & 0.265 & 0.207 & 0.119 & 0.134 & 0.114 & 0.106 & 0.077 \\
13 & 1.375 & 0.4

In [ ]:
from FZCP.test import test_alg3

test_alg3(df, 1e-5, global_lip=True, repeat=10)

In [ ]:
from FZCP.test import test_alg3

test_alg3(df, 1e-5, global_lip=False, repeat=1)

In [ ]:
from FZCP.test import test_casado

test_casado(df, 1e-5, repeat=100)

In [8]:
for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    # #
    # T1 = time.perf_counter()
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_psl_points_list']))
    # print('Piyavskii, Lipschitz constant:',
    #       sfzcp.new_method(prob, symm=True, epsilon=1e-5, global_lipschitz_interval=False, estimator=1))
    # T2 = time.perf_counter()
    # print('time=', T2 - T1)
    #
    # T1 = time.perf_counter()
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    # print('Piyavskii, Lipschitz interval:',
    #       sfzcp.new_method(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1))
    # T2 = time.perf_counter()
    # print('time=', T2 - T1)
    #
    # T1 = time.perf_counter()
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_lip_points_list']))
    # print('PSQE, Lipschitz constant:',
    #       sfzcp.new_method(prob, symm=True, epsilon=1e-5, global_lipschitz_interval=False, estimator=2))
    # T2 = time.perf_counter()
    # print('time=', T2 - T1)

    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
                              lambda x: log_point(x, points_db[test.Index]['bnb2_lipint_points_list']))

    T1 = time.perf_counter()
    print('PSQE, Lipschitz interval:',
          sfzcp.new_method(prob, symm=True, epsilon=1e-7, global_lipschitz_interval=True, estimator=2, reduction=0))
    T2 = time.perf_counter()
    print('time=', T2 - T1)

    T1 = time.perf_counter()
    print('Casado:',
          sfzcp.cas(prob=prob, sym=True, epsilon=1e-7))
    T2 = time.perf_counter()
    print('time=', T2 - T1)
    # T1 = time.perf_counter()
    # print('Casado:', sfzcp.cas(prob, epsilon=1e-5))
    # T2 = time.perf_counter()
    # print('time=', T2 - T1)

1
error d>b
a= 3.0116907417774206 b= 3.0116908431053164 c= 3.0116908320426967 d= 3.0116908444430215 delt= 1.2400324568309995e-08 fa= 1.412606653872217e-07 fb= -3.4777202362334947e-07 alp= -3.4459101490553135 beta= 3.4459101490553135
PSQE, Lipschitz interval: TestResult(nsteps=41, first_crossing_zero_point=[3.0116907417774206])
time= 0.000310583011014387
Casado: TestResult(nsteps=26, first_crossing_zero_point=[3.011690792344809])
time= 0.0003095419961027801
2
PSQE, Lipschitz interval: TestResult(nsteps=31, first_crossing_zero_point=[])
time= 0.0002435839851386845
Casado: TestResult(nsteps=1, first_crossing_zero_point=[])
time= 1.8041988369077444e-05
3
PSQE, Lipschitz interval: TestResult(nsteps=42, first_crossing_zero_point=[1.1747961461544036])
time= 0.0002727080136537552
Casado: TestResult(nsteps=33, first_crossing_zero_point=[1.1747961554858224])
time= 0.0002772080188151449
4
PSQE, Lipschitz interval: TestResult(nsteps=50, first_crossing_zero_point=[2.9609137624502173])
time= 0.00036

In [ ]:
print('test.Index,PC_N,PI_N,QC_N,QI_N,PC_R,PI_R,QC_R,QI_R')
eps = 1e-7
min_list = (float('inf'),) * 8
max_list = (0,) * 8
sum_list = (0,) * 8
for test in df.itertuples():
    points_db[test.Index] = {'bnb2_pslint_points_list': []}
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
                              lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    T1 = time.perf_counter()
    PC_N = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=False).nsteps
    T2 = time.perf_counter()
    time_PC_N = T2 - T1
    min_list[0] = min(min_list[0], T2 - T1)
    max_list[0] = max(max_list[0], T2 - T1)
    sum_list[0] += T2 - T1
    T1 = time.perf_counter()
    PI_N = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=False).nsteps
    T2 = time.perf_counter()
    time_PI_N = T2 - T1
    min_list[1] = min(min_list[1], T2 - T1)
    max_list[1] = max(max_list[1], T2 - T1)
    sum_list[1] += T2 - T1
    T1 = time.perf_counter()
    QC_N = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=False).nsteps
    T2 = time.perf_counter()
    time_QC_N = T2 - T1
    min_list[2] = min(min_list[2], T2 - T1)
    max_list[2] = max(max_list[2], T2 - T1)
    sum_list[2] += T2 - T1
    T1 = time.perf_counter()
    QI_N = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=False).nsteps
    T2 = time.perf_counter()
    time_QI_N = T2 - T1
    min_list[3] = min(min_list[3], T2 - T1)
    max_list[3] = max(max_list[3], T2 - T1)
    sum_list[3] += T2 - T1
    T1 = time.perf_counter()
    PC_R = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=True).nsteps
    T2 = time.perf_counter()
    time_PC_R = T2 - T1
    min_list[4] = min(min_list[4], T2 - T1)
    max_list[4] = max(max_list[4], T2 - T1)
    sum_list[4] += T2 - T1
    T1 = time.perf_counter()
    PI_R = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=True).nsteps
    T2 = time.perf_counter()
    time_PI_R = T2 - T1
    min_list[5] = min(min_list[5], T2 - T1)
    max_list[5] = max(max_list[5], T2 - T1)
    sum_list[5] += T2 - T1
    T1 = time.perf_counter()
    QC_R = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=True).nsteps
    T2 = time.perf_counter()
    time_QC_R = T2 - T1
    min_list[6] = min(min_list[6], T2 - T1)
    max_list[6] = max(max_list[6], T2 - T1)
    sum_list[6] += T2 - T1
    T1 = time.perf_counter()
    QI_R = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=True).nsteps
    T2 = time.perf_counter()
    time_QI_R = T2 - T1
    min_list[7] = min(min_list[7], T2 - T1)
    max_list[7] = max(max_list[7], T2 - T1)
    sum_list[7] += T2 - T1

    # print('%s & %d & %d & %d & %d & %d & %d & %d & %d \\\\' %(test.Index,PC_N,PI_N,QC_N,QI_N,PC_R,PI_R,QC_R,QI_R))
    print('%s & %.5f & %.5f & %.5f & %.5f & %.5f & %.5f & %.5f & %.5f \\\\' % (
        test.Index, time_PC_N, time_PI_N, time_QC_N, time_QI_N, time_PC_R, time_PI_R, time_QC_R, time_QI_R))

In [ ]:
print('test.Index,PC_N,PI_N,QC_N,QI_N,PC_R,PI_R,QC_R,QI_R')
eps = 1e-7
for test in df.itertuples():
    points_db[test.Index] = {'bnb2_pslint_points_list': []}
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
                              lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    PC_N = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=False).nsteps

    PI_N = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=False).nsteps
    QC_N = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=False).nsteps
    QI_N = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=False).nsteps

    PC_R = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=True).nsteps
    PI_R = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=1,
                            reduction=True).nsteps
    QC_R = sfzcp.new_method(prob, symm=True, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=True).nsteps
    QI_R = sfzcp.new_method(prob, symm=False, epsilon=eps, global_lipschitz_interval=True, estimator=2,
                            reduction=True).nsteps

    print('%s & %d & %d & %d & %d & %d & %d & %d & %d \\\\' % (
        test.Index, PC_N, PI_N, QC_N, QI_N, PC_R, PI_R, QC_R, QI_R))
    # P: Piyavskii estimator
    # Q: Piecewise smooth quadratic estimator
    # C: Lipschitz constant
    # I: Lipschitz interval
    # N: without reduction
    # R: with reduction

In [ ]:
def vis_trials(points_list, num_series, base_level):
    poix = np.array(list(set(points_list[::skip])))
    poiy = np.full(len(poix), base_level + num_series * -0.5)
    up.plot_points(poix, poiy, num_series)

In [ ]:
for test in df.itertuples():
    print(test)
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    up.plot_problem(prob, {'function': 'b-'}, legend=legend_size)
    vis_trials(points_db[test.Index]['bnb2_psl_points_list'], 0, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['bnb2_pslint_points_list'], 1, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['bnb2_lip_points_list'], 2, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['bnb2_lipint_points_list'], 3, plt.ylim()[0] + legend_size)
    # vis_trials(points_db[test.Index]['psl_lipint_points_list'], 4, plt.ylim()[0] + legend_size)
    # vis_trials(points_db[test.Index]['psl_lip_points_list'], 5, plt.ylim()[0] + legend_size)
    #     print("ylim = ", plt.ylim()[0])
    file_name = test.Index + '.png'
    plt.savefig(file_name, dpi=500)
    plt.show()